# Parselmouth, a tutorial

## Introduction

### Installation

Install Parselmouth by installing the `praat-parselmouth` package from PyPI.

Do not - do or do not? I repeat, do not! - install `parselmouth`, as it is another package. _(Sorry for the confusion.)_

In [ ]:
!pip install praat-parselmouth

And these are some other things you should have installed to run demo:

In [ ]:
!pip install numpy pandas matplotlib seaborn

### Let's get started

In [ ]:
import parselmouth

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
parselmouth.__version__

In [ ]:
parselmouth.PRAAT_VERSION, parselmouth.PRAAT_VERSION_DATE

### What's to come?

- That one example from before
- Fantastic Praat objects and how to plot them
- Accessing "raw" Praat commands
- Running Praat scripts

**Main goal: how to map a Praat workflow to Parselmouth**

_(in other words: the examples might seem a bit artificial/simplistic, but try to find the principles)_

### Synchronisation point

- Questions/issues/... ?
- https://parselmouth.readthedocs.io/

## That one example from before

In [ ]:
import parselmouth

def extract_info(sound):
    pitch = sound.to_pitch(time_step=0.001, pitch_ceiling=300)
    intensity = sound.to_intensity(75, 0.001, subtract_mean=False)

    print("Here are the results:")
    for i in range(int((sound.tmax - sound.tmin) / 0.01)):
        time = sound.tmin + (i + 1) * 0.01
        p_value = pitch.get_value_at_time(time)
        i_value = intensity.get_value(time)
        print("{:.2f} {:.3f} {:.3f}".format(time, p_value, i_value))

extract_info(parselmouth.Sound("data/the_north_wind_and_the_sun.wav"))

### Let's break that up into pieces

In [ ]:
sound = parselmouth.Sound("data/the_north_wind_and_the_sun.wav")

In [ ]:
sound

In [ ]:
print(sound)

### Seems familiar?

![praat_objects_and_info.png](images/praat_objects_and_info.png)

### Intensity and Pitch

In [ ]:
pitch = sound.to_pitch(time_step=0.001, pitch_ceiling=300)
pitch

In [ ]:
pitch.get_value_at_time(0.5)

![praat_pitch_value.png](images/praat_pitch_get_value.png)

![praat_pitch_value.png](images/praat_pitch_value.png)

In [ ]:
intensity = sound.to_intensity(75, 0.001, subtract_mean=False)
intensity

In [ ]:
intensity.get_value(0.5)

In [ ]:
sound.tmin, sound.tmax

### And that's all Python-Praat interaction

The rest is Python!

In [ ]:
import parselmouth

def extract_info(sound):
    pitch = sound.to_pitch(time_step=0.001, pitch_ceiling=300)
    intensity = sound.to_intensity(75, 0.001, subtract_mean=False)

    print("Here are the results:")
    for i in range(int((sound.tmax - sound.tmin) / 0.01)):
        time = sound.tmin + (i + 1) * 0.01
        p_value = pitch.get_value_at_time(time)
        i_value = intensity.get_value(time)
        print("{:.2f} {:.3f} {:.3f}".format(time, p_value, i_value))

extract_info(parselmouth.Sound("data/the_north_wind_and_the_sun.wav"))

### Synchronisation point

- https://parselmouth.readthedocs.io/en/stable/api_reference.html#parselmouth.Sound
- Change pitch/intensity extraction step to auto
- Extract formant analysis and print first 2 formants

## Fantastic Praat objects and how to plot them

Two goals at once:
1. How to make nice plots and integrate Praat (Parselmouth) into Python plotting libraries
2. How to access the actual data to be plotting **(!)**

In [ ]:
sound = parselmouth.Sound("data/the_north_wind_and_the_sun.wav")
sound

In [ ]:
help(sound.get_value)

In [ ]:
sound.n_channels

In [ ]:
sound.get_value(0.5, 1)

In [ ]:
sound.values

In [ ]:
type(sound.values), sound.values.shape

In [ ]:
sound.values[0,:-100:2]

In [ ]:
parselmouth.Sound.__mro__  # Subclass of parselmouth.Matrix

In [ ]:
from IPython.display import Audio

Audio(sound.values[0,:], rate=sound.sampling_frequency)

In [ ]:
def audio_player(sound):
    return Audio(sound.values[0,:], rate=sound.sampling_frequency)

### Plotting the waveform

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [9, 6]
plt.rcParams['figure.dpi'] = 100

In [ ]:
sound.values[0,:]

In [ ]:
sound.x1 + np.arange(sound.nx) * sound.dx

In [ ]:
sound.xs()

In [ ]:
fig, ax = plt.subplots()
ax.plot(sound.xs(), sound.values[0,:])

In [ ]:
fig, ax = plt.subplots()
ax.plot(sound.xs(), sound.values[0,:], color='pink', linewidth=0.5)
ax.set_xlim(sound.xmin, sound.xmax)
ax.set_ylim(-1, 1)

In [ ]:
sun_sound = sound.extract_part(from_time=0.9, preserve_times=True)
audio_player(sun_sound)

In [ ]:
def plot_waveform(ax, sound, **kwargs):
    max_abs_val = np.max(np.abs(sun_sound.values))
    ax.plot(sun_sound.xs(), sun_sound.values[0,:], **kwargs)
    ax.set_xlim(sound.xmin, sound.xmax)
    ax.set_ylim(-1.2 * max_abs_val, 1.2 * max_abs_val)
    ax.set_xlabel("time (s)")
    ax.set_ylabel("amplitude")

fig, ax = plt.subplots()
plot_waveform(ax, sun_sound, linestyle='--', color='darkolivegreen')

### And now a spectrogram

In [ ]:
spectrogram = sound.to_spectrogram(window_length=0.03)
spectrogram

In [ ]:
help(sound.to_spectrogram)

In [ ]:
isinstance(spectrogram, parselmouth.Matrix), issubclass(parselmouth.Spectrogram, parselmouth.Matrix)

In [ ]:
spectrogram.values

In [ ]:
spectrogram.values.shape, spectrogram.values.size

In [ ]:
spectrogram.values.base

In [ ]:
spectrogram.xs().shape, spectrogram.xs()

In [ ]:
spectrogram.x_grid().shape, spectrogram.x_grid()

In [ ]:
fig, ax = plt.subplots()
ax.pcolormesh(spectrogram.x_grid(), spectrogram.y_grid(), spectrogram.values)

In [ ]:
fig, ax = plt.subplots()
ax.pcolormesh(spectrogram.x_grid(), spectrogram.y_grid(), 10 * np.log10(spectrogram.values))

In [ ]:
def draw_spectrogram(ax, spectrogram, dynamic_range=70):
    X, Y = spectrogram.x_grid(), spectrogram.y_grid()
    sg_db = 10 * np.log10(spectrogram.values)
    ax.pcolormesh(X, Y, sg_db, vmin=sg_db.max() - dynamic_range, cmap='afmhot')
    ax.set_ylim(spectrogram.ymin, spectrogram.ymax)
    ax.set_xlabel("time (s)")
    ax.set_ylabel("frequency (Hz)")

fig, ax = plt.subplots()
draw_spectrogram(ax, spectrogram)

In [ ]:
fig, ax = plt.subplots()
pre_emphasized_sound = sound.copy()
pre_emphasized_sound.pre_emphasize()
draw_spectrogram(ax, pre_emphasized_sound.to_spectrogram(window_length=0.005, maximum_frequency=16000))

### And the pitch contour

In [ ]:
pitch = sound.to_pitch()
pitch

In [ ]:
hasattr(pitch, 'values')

In [ ]:
pitch.get_frame(20)

In [ ]:
pitch.get_frame(20).candidates

In [ ]:
pitch.get_frame(20).candidates[0].frequency

![praat_pitch_candidates.png](images/praat_pitch_candidates.png)

In [ ]:
pitch.selected_array

In [ ]:
def draw_pitch(ax, pitch):
    # Extract selected pitch contour, and
    # replace unvoiced samples by NaN to not plot
    pitch_values = pitch.selected_array['frequency']
    pitch_values[pitch_values==0] = np.nan
    ax.plot(pitch.xs(), pitch_values, 'o', markersize=5, color='w')
    ax.plot(pitch.xs(), pitch_values, 'o', markersize=2)
    ax.grid(False)
    ax.set_ylim(0, pitch.ceiling)
    ax.set_ylabel("fundamental frequency (Hz)")

In [ ]:
fig, ax = plt.subplots()
draw_spectrogram(ax, spectrogram)
ax_twin = ax.twinx()
draw_pitch(ax_twin, pitch)
ax.set_xlim(sound.xmin, sound.xmax)

### Synchronisation point

- https://parselmouth.readthedocs.io/en/stable/examples/plotting.html
- Play around with styles and colors and colormaps
- Plot a formant track

## Accessing "raw" Praat commands

Creating a Python interface is quite a bit of work. See source ;-)

What to do when no Python class exists?

In [ ]:
manipulation = parselmouth.praat.call(sound, "To Manipulation", 0.01, 75, 600)
manipulation

In [ ]:
type(manipulation), type(sound)

In [ ]:
parselmouth.Sound.__mro__

In [ ]:
help(parselmouth.Data)

In [ ]:
manipulation.class_name

`parselmouth.Data` is the base class of all Praat objects (actually `parselmouth.Thing`; it's complicated).

When the full type is not exposed in Parselmouth, you get a `parselmouth.Data`.

In [ ]:
?parselmouth.praat.call

https://parselmouth.readthedocs.io/en/stable/api_reference.html#parselmouth.praat.call